# Model Free Monte Carlo : 


Monte Carlo Methods can learn directly from episodes. It is a model free approach, so it is usefull when we do not have knowledge about the environment (MDP).

It needs to learn from an entire episode. To evaluate the value of each state, you have to compute the mean return (rewards). The Monte Carlo methods will be used to evaluate a policy (Policy Evaluation), then the policy can be improved with Policy Improvement (greedy approach). In pratice you will save every state, rewards of each episode and then compute the value of the state iteratively based on the formula : 
$V^\pi\left(s_{t}\right) \leftarrow V^\pi\left(s_{t}\right)+\alpha\left(r_{k}+r_{k+1}+\cdots+r_{N}-V^\pi\left(s_{t}\right)\right) = V^\pi\left(s_{t}\right)+\alpha\left(G_k - V^\pi\left(s_{t}\right)\right) $

$G_k = \gamma G_{k+1} + r_{k+1}$ with $G_k$ the cumulative rewards.


<img src="img/MonteCarloControl.png" >   

# Monte Carlo Control

In [1]:
import gym
import numpy as np

In [2]:
env = gym.make("MountainCar-v0")
env._max_episode_steps = 5000
"""
Actions:
        Type: Discrete(3)
        Num    Action
        0      Accelerate to the Left
        1      Don't accelerate
        2      Accelerate to the Right
"""

def convert2discrete(observations, N=30):
    """
    Observation:
        Type: Box(2)
        Num    Observation               Min            Max
        0      Car Position              -1.2           0.6
        1      Car Velocity              -0.07          0.07

    """
    position = observations[0]
    velocity = observations[1]
    
    min_pos = -1.2
    max_pos = 0.6
    step_pos = (max_pos - min_pos)/(N-1)
    
    min_vel = -0.07
    max_vel = 0.07
    step_vel = (max_vel - min_vel)/(N-1)
    
    new_position = (position - min_pos)//step_pos
    
    new_velocity = (velocity - min_vel)//step_vel
    
    return int(new_position), int(new_velocity)
    
    
    


epsilon=1.0
counts_steps = []
success = 0.
numberExp = 10000
discount = 0.95
discr = 20
q = np.zeros((discr,discr,3))
N = np.zeros((discr, discr,3))
R = np.zeros((discr, discr,3))

In [3]:
for experiment in range(0, numberExp):
    Rewards = []
    S = []
    observation = env.reset()
    
    new_p, new_v = convert2discrete(observation, N=discr)
    # Generate episode and save state, action
    for curr_step in range(0, env._max_episode_steps):
        
        ## Select an action 
        if np.random.uniform()<=epsilon:
            action = env.action_space.sample() # your agent here (this takes random actions)
        else:
            action = np.argmax(q[new_p, new_v,:])
            
        # get new observation
        new_observation, reward, done, info = env.step(action)
        
        # save state
        S.append( (new_p, new_v, action) )
        
        
        Rewards.append(reward)
        # discretise states
        new_p, new_v = convert2discrete(new_observation, N=discr)
        
        if done:
            #print(info)
            if 'TimeLimit.truncated' not in info:
                Rewards[-1] = env._max_episode_steps
                success += 1
                if success > 10:
                    epsilon = max(epsilon * 0.99,0.05)
            break
    # update value
    i = len(S) - 1
    G = 0
    while i >= 0:
        G = discount * G + Rewards[i]
        if S[i] not in set(S[:i]):
            N[S[i][0], S[i][1], S[i][2]] += 1 # counter needed for computing the average later
            R[S[i][0], S[i][1], S[i][2]] += G # Cumulative reward with discount save depending of the state
            q[S[i][0], S[i][1], S[i][2]] = R[S[i][0], S[i][1], S[i][2]] / N[S[i][0], S[i][1], S[i][2]] # update the q value
        i -=1

    counts_steps.append(curr_step)
    print(f"rate of success : {success}/{experiment+1}, epsilon={epsilon}, num_steps:{curr_step}")
    if np.mean(counts_steps[-100:])<2500:
        env._max_episode_steps = 1500
    elif np.mean(counts_steps[-100:])<1000:
        env._max_episode_steps = 800
    elif np.mean(counts_steps[-100:])<500:
        env._max_episode_steps = 400
    elif np.mean(counts_steps[-100:])<300:
        env._max_episode_steps = 200
env.close()
np.save("monte_carlo_moutaincar", q)

rate of success : 0.0/1, epsilon=1.0, num_steps:4999
rate of success : 0.0/2, epsilon=1.0, num_steps:4999
rate of success : 0.0/3, epsilon=1.0, num_steps:4999
rate of success : 0.0/4, epsilon=1.0, num_steps:4999
rate of success : 0.0/5, epsilon=1.0, num_steps:4999
rate of success : 0.0/6, epsilon=1.0, num_steps:4999
rate of success : 0.0/7, epsilon=1.0, num_steps:4999
rate of success : 0.0/8, epsilon=1.0, num_steps:4999
rate of success : 0.0/9, epsilon=1.0, num_steps:4999
rate of success : 0.0/10, epsilon=1.0, num_steps:4999
rate of success : 0.0/11, epsilon=1.0, num_steps:4999
rate of success : 0.0/12, epsilon=1.0, num_steps:4999
rate of success : 0.0/13, epsilon=1.0, num_steps:4999
rate of success : 0.0/14, epsilon=1.0, num_steps:4999
rate of success : 0.0/15, epsilon=1.0, num_steps:4999
rate of success : 1.0/16, epsilon=1.0, num_steps:2928
rate of success : 1.0/17, epsilon=1.0, num_steps:4999
rate of success : 1.0/18, epsilon=1.0, num_steps:4999
rate of success : 1.0/19, epsilon=1.0

rate of success : 10.0/152, epsilon=1.0, num_steps:4999
rate of success : 10.0/153, epsilon=1.0, num_steps:4999
rate of success : 11.0/154, epsilon=0.99, num_steps:3760
rate of success : 11.0/155, epsilon=0.99, num_steps:4999
rate of success : 11.0/156, epsilon=0.99, num_steps:4999
rate of success : 11.0/157, epsilon=0.99, num_steps:4999
rate of success : 12.0/158, epsilon=0.9801, num_steps:3508
rate of success : 12.0/159, epsilon=0.9801, num_steps:4999
rate of success : 12.0/160, epsilon=0.9801, num_steps:4999
rate of success : 12.0/161, epsilon=0.9801, num_steps:4999
rate of success : 12.0/162, epsilon=0.9801, num_steps:4999
rate of success : 12.0/163, epsilon=0.9801, num_steps:4999
rate of success : 12.0/164, epsilon=0.9801, num_steps:4999
rate of success : 12.0/165, epsilon=0.9801, num_steps:4999
rate of success : 12.0/166, epsilon=0.9801, num_steps:4999
rate of success : 12.0/167, epsilon=0.9801, num_steps:4999
rate of success : 12.0/168, epsilon=0.9801, num_steps:4999
rate of suc

rate of success : 33.0/272, epsilon=0.7936142836436553, num_steps:4777
rate of success : 33.0/273, epsilon=0.7936142836436553, num_steps:4999
rate of success : 34.0/274, epsilon=0.7856781408072188, num_steps:4191
rate of success : 34.0/275, epsilon=0.7856781408072188, num_steps:4999
rate of success : 34.0/276, epsilon=0.7856781408072188, num_steps:4999
rate of success : 35.0/277, epsilon=0.7778213593991465, num_steps:1122
rate of success : 35.0/278, epsilon=0.7778213593991465, num_steps:4999
rate of success : 35.0/279, epsilon=0.7778213593991465, num_steps:4999
rate of success : 36.0/280, epsilon=0.7700431458051551, num_steps:3613
rate of success : 37.0/281, epsilon=0.7623427143471035, num_steps:997
rate of success : 37.0/282, epsilon=0.7623427143471035, num_steps:4999
rate of success : 38.0/283, epsilon=0.7547192872036325, num_steps:1835
rate of success : 39.0/284, epsilon=0.7471720943315961, num_steps:4897
rate of success : 40.0/285, epsilon=0.7397003733882802, num_steps:3252
rate of

rate of success : 102.0/387, epsilon=0.39667780642202527, num_steps:1499
rate of success : 102.0/388, epsilon=0.39667780642202527, num_steps:1499
rate of success : 102.0/389, epsilon=0.39667780642202527, num_steps:1499
rate of success : 102.0/390, epsilon=0.39667780642202527, num_steps:1499
rate of success : 102.0/391, epsilon=0.39667780642202527, num_steps:1499
rate of success : 102.0/392, epsilon=0.39667780642202527, num_steps:1499
rate of success : 102.0/393, epsilon=0.39667780642202527, num_steps:1499
rate of success : 102.0/394, epsilon=0.39667780642202527, num_steps:1499
rate of success : 102.0/395, epsilon=0.39667780642202527, num_steps:1499
rate of success : 102.0/396, epsilon=0.39667780642202527, num_steps:1499
rate of success : 102.0/397, epsilon=0.39667780642202527, num_steps:1499
rate of success : 102.0/398, epsilon=0.39667780642202527, num_steps:1499
rate of success : 102.0/399, epsilon=0.39667780642202527, num_steps:1499
rate of success : 102.0/400, epsilon=0.396677806422

rate of success : 103.0/502, epsilon=0.392711028357805, num_steps:1499
rate of success : 103.0/503, epsilon=0.392711028357805, num_steps:1499
rate of success : 103.0/504, epsilon=0.392711028357805, num_steps:1499
rate of success : 103.0/505, epsilon=0.392711028357805, num_steps:1499
rate of success : 103.0/506, epsilon=0.392711028357805, num_steps:1499
rate of success : 103.0/507, epsilon=0.392711028357805, num_steps:1499
rate of success : 103.0/508, epsilon=0.392711028357805, num_steps:1499
rate of success : 103.0/509, epsilon=0.392711028357805, num_steps:1499
rate of success : 103.0/510, epsilon=0.392711028357805, num_steps:1499
rate of success : 103.0/511, epsilon=0.392711028357805, num_steps:1499
rate of success : 103.0/512, epsilon=0.392711028357805, num_steps:1499
rate of success : 103.0/513, epsilon=0.392711028357805, num_steps:1499
rate of success : 103.0/514, epsilon=0.392711028357805, num_steps:1499
rate of success : 103.0/515, epsilon=0.392711028357805, num_steps:1499
rate o

...

rate of success : 8662.0/9249, epsilon=0.05, num_steps:159
rate of success : 8663.0/9250, epsilon=0.05, num_steps:250
rate of success : 8664.0/9251, epsilon=0.05, num_steps:154
rate of success : 8665.0/9252, epsilon=0.05, num_steps:234
rate of success : 8666.0/9253, epsilon=0.05, num_steps:158
rate of success : 8667.0/9254, epsilon=0.05, num_steps:170
rate of success : 8668.0/9255, epsilon=0.05, num_steps:167
rate of success : 8669.0/9256, epsilon=0.05, num_steps:161
rate of success : 8670.0/9257, epsilon=0.05, num_steps:156
rate of success : 8671.0/9258, epsilon=0.05, num_steps:233
rate of success : 8672.0/9259, epsilon=0.05, num_steps:154
rate of success : 8673.0/9260, epsilon=0.05, num_steps:167
rate of success : 8674.0/9261, epsilon=0.05, num_steps:243
rate of success : 8675.0/9262, epsilon=0.05, num_steps:236
rate of success : 8676.0/9263, epsilon=0.05, num_steps:250
rate of success : 8677.0/9264, epsilon=0.05, num_steps:159
rate of success : 8678.0/9265, epsilon=0.05, num_steps:1

rate of success : 8821.0/9408, epsilon=0.05, num_steps:244
rate of success : 8822.0/9409, epsilon=0.05, num_steps:175
rate of success : 8823.0/9410, epsilon=0.05, num_steps:243
rate of success : 8824.0/9411, epsilon=0.05, num_steps:233
rate of success : 8825.0/9412, epsilon=0.05, num_steps:240
rate of success : 8826.0/9413, epsilon=0.05, num_steps:157
rate of success : 8827.0/9414, epsilon=0.05, num_steps:234
rate of success : 8828.0/9415, epsilon=0.05, num_steps:175
rate of success : 8829.0/9416, epsilon=0.05, num_steps:168
rate of success : 8830.0/9417, epsilon=0.05, num_steps:235
rate of success : 8831.0/9418, epsilon=0.05, num_steps:245
rate of success : 8832.0/9419, epsilon=0.05, num_steps:164
rate of success : 8833.0/9420, epsilon=0.05, num_steps:151
rate of success : 8834.0/9421, epsilon=0.05, num_steps:247
rate of success : 8835.0/9422, epsilon=0.05, num_steps:174
rate of success : 8836.0/9423, epsilon=0.05, num_steps:171
rate of success : 8837.0/9424, epsilon=0.05, num_steps:2

rate of success : 8982.0/9569, epsilon=0.05, num_steps:188
rate of success : 8983.0/9570, epsilon=0.05, num_steps:250
rate of success : 8984.0/9571, epsilon=0.05, num_steps:159
rate of success : 8985.0/9572, epsilon=0.05, num_steps:237
rate of success : 8986.0/9573, epsilon=0.05, num_steps:250
rate of success : 8987.0/9574, epsilon=0.05, num_steps:149
rate of success : 8988.0/9575, epsilon=0.05, num_steps:230
rate of success : 8989.0/9576, epsilon=0.05, num_steps:160
rate of success : 8990.0/9577, epsilon=0.05, num_steps:242
rate of success : 8991.0/9578, epsilon=0.05, num_steps:255
rate of success : 8992.0/9579, epsilon=0.05, num_steps:161
rate of success : 8993.0/9580, epsilon=0.05, num_steps:161
rate of success : 8994.0/9581, epsilon=0.05, num_steps:161
rate of success : 8995.0/9582, epsilon=0.05, num_steps:152
rate of success : 8996.0/9583, epsilon=0.05, num_steps:161
rate of success : 8997.0/9584, epsilon=0.05, num_steps:166
rate of success : 8998.0/9585, epsilon=0.05, num_steps:1

rate of success : 9144.0/9731, epsilon=0.05, num_steps:164
rate of success : 9145.0/9732, epsilon=0.05, num_steps:158
rate of success : 9146.0/9733, epsilon=0.05, num_steps:167
rate of success : 9147.0/9734, epsilon=0.05, num_steps:164
rate of success : 9148.0/9735, epsilon=0.05, num_steps:245
rate of success : 9149.0/9736, epsilon=0.05, num_steps:163
rate of success : 9150.0/9737, epsilon=0.05, num_steps:254
rate of success : 9151.0/9738, epsilon=0.05, num_steps:163
rate of success : 9152.0/9739, epsilon=0.05, num_steps:239
rate of success : 9153.0/9740, epsilon=0.05, num_steps:156
rate of success : 9154.0/9741, epsilon=0.05, num_steps:154
rate of success : 9155.0/9742, epsilon=0.05, num_steps:166
rate of success : 9156.0/9743, epsilon=0.05, num_steps:167
rate of success : 9157.0/9744, epsilon=0.05, num_steps:161
rate of success : 9158.0/9745, epsilon=0.05, num_steps:155
rate of success : 9159.0/9746, epsilon=0.05, num_steps:232
rate of success : 9160.0/9747, epsilon=0.05, num_steps:1

rate of success : 9304.0/9891, epsilon=0.05, num_steps:251
rate of success : 9305.0/9892, epsilon=0.05, num_steps:167
rate of success : 9306.0/9893, epsilon=0.05, num_steps:166
rate of success : 9307.0/9894, epsilon=0.05, num_steps:189
rate of success : 9308.0/9895, epsilon=0.05, num_steps:246
rate of success : 9309.0/9896, epsilon=0.05, num_steps:160
rate of success : 9310.0/9897, epsilon=0.05, num_steps:174
rate of success : 9311.0/9898, epsilon=0.05, num_steps:236
rate of success : 9312.0/9899, epsilon=0.05, num_steps:158
rate of success : 9313.0/9900, epsilon=0.05, num_steps:156
rate of success : 9314.0/9901, epsilon=0.05, num_steps:227
rate of success : 9315.0/9902, epsilon=0.05, num_steps:169
rate of success : 9316.0/9903, epsilon=0.05, num_steps:154
rate of success : 9317.0/9904, epsilon=0.05, num_steps:238
rate of success : 9318.0/9905, epsilon=0.05, num_steps:153
rate of success : 9319.0/9906, epsilon=0.05, num_steps:168
rate of success : 9320.0/9907, epsilon=0.05, num_steps:1

In [4]:
len(S), len(Rewards)

(161, 161)

In [5]:
counts_steps[-100:]

[227,
 169,
 154,
 238,
 153,
 168,
 174,
 160,
 159,
 238,
 156,
 240,
 168,
 162,
 173,
 169,
 247,
 162,
 164,
 169,
 249,
 167,
 149,
 253,
 163,
 232,
 182,
 170,
 240,
 153,
 159,
 237,
 172,
 160,
 170,
 181,
 236,
 164,
 152,
 249,
 168,
 159,
 174,
 163,
 239,
 164,
 151,
 244,
 157,
 166,
 149,
 166,
 171,
 174,
 241,
 241,
 156,
 161,
 247,
 251,
 246,
 168,
 236,
 235,
 256,
 247,
 155,
 170,
 158,
 161,
 166,
 156,
 251,
 173,
 168,
 169,
 251,
 161,
 234,
 242,
 170,
 172,
 165,
 154,
 235,
 170,
 152,
 152,
 172,
 173,
 165,
 249,
 256,
 168,
 160,
 158,
 165,
 163,
 148,
 160]

In [4]:
from moviepy.editor import ImageSequenceClip
def test_model(env, q, N, gif_name=None):
    frames = []
    observation = env.reset()
    new_p, new_v = convert2discrete(observation, N=discr)
    for curr_step in range(0, env._max_episode_steps):
        
        frame = env.render(mode='rgb_array')
        frames.append(frame) 
        action = np.argmax(q[new_p, new_v,:])
        new_observation, reward, done, info = env.step(action)
        new_p, new_v = convert2discrete(new_observation, N=N)
        if done:
            print(f"success : {curr_step}")
            break
    env.close()
    if gif_name is not None:
        clip = ImageSequenceClip(frames, fps=20)
        clip.write_gif(gif_name, fps=20)

q = np.load("monte_carlo_moutaincar.npy")
test_model(env,q,discr, "monte_carlo_moutaincar.gif")
        

t:   7%|████▍                                                              | 11/165 [00:00<00:01, 108.24it/s, now=None]

success : 164
MoviePy - Building file monte_carlo_moutaincar.gif with imageio.
